In [1]:
from google.colab import drive
drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/new_project/project2'
traindata_path = '/content/drive/MyDrive/new_project/project2/traindata/문서요약 텍스트/'
rawdata_path = '/content/drive/MyDrive/new_project/project2/rawdata/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
np.random.seed(seed=0)

In [32]:
data = pd.read_csv(f"{traindata_path}/kobart_train.tsv", nrows = 10000, sep = '\t')
print('전체 리뷰 개수 :',(len(data)))

전체 리뷰 개수 : 10000


In [33]:
data.columns = ['text', 'headlines']

In [34]:
data.head()

,text,headlines
0,ha당 조사료 400만원…작물별 차등 지원 전라남도가 쌀 과잉문제를 근본적으로 해결...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 여수시는 원도심 일대에서 추진된 컬러...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발 전남드래곤즈(사장 신승재)는 지난 4일 구...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 등 지역특화작목 광양시는 오는 11일부터 24일까지 ...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 타이완의 크루즈관광객 4000...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


In [35]:
print('Text 열에서 중복을 배제한 유일한 샘플의 수 :', data['text'].nunique())
print('Summary 열에서 중복을 배제한 유일한 샘플의 수 :', data['headlines'].nunique())

Text 열에서 중복을 배제한 유일한 샘플의 수 : 9996
Summary 열에서 중복을 배제한 유일한 샘플의 수 : 9999


In [36]:
# text 열에서 중복인 내용이 있다면 중복 제거
data.drop_duplicates(subset=['text'], inplace=True)
data.drop_duplicates(subset=['headlines'], inplace=True)
print("전체 샘플수 :", len(data))

전체 샘플수 : 9995


In [37]:
print(data.isnull().sum())

text         0
headlines    0
dtype: int64


In [38]:
# Null 값을 가진 샘플 제거
data.dropna(axis=0, inplace=True)
print('전체 샘플수 :',(len(data)))

전체 샘플수 : 9995


In [39]:
with open(f'{rawdata_path}stopwords_korean.txt', 'r') as file:
    stopwords_korean = file.readlines()

# Remove whitespace and newlines from the stopwords
stopwords_korean = [word.strip() for word in stopwords_korean]
stop_words = set(stopwords_korean)
len(stop_words)

595

In [40]:
def preprocess_sentence(sentence, remove_stopwords = True):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열  제거 Ex) my husband (and myself) for => my husband for
    sentence = re.sub(r'\[[^]]*\)', '', sentence) # 괄호( []  )로 닫힌 문자열  제거 Ex) my husband (and myself) for => my husband for
    sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거

    #### 이메일 형식 제거
    sentence = re.sub(r'\w+@\w+\.\w+', '', sentence)

    ## 기자
    # sentence = re.sub(r'\*{3}\s*기자\s*', '', sentence)
    # sentence = re.sub(r'\*{2}\s*기자\s*', '', sentence)

    # 불용어 제거 (Text)
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stop_words if len(word) > 1)
    # 불용어 미제거 (Summary)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens

In [41]:
# data['text'] 부분 전처리 진행

clean_text = []
for s in data['text']:
  clean_text.append(preprocess_sentence(s))

print('Text 전처리 후 결과 : ', clean_text[:5])

Text 전처리 후 결과 :  ['ha당 조사료 400만원…작물별 차등 지원 전라남도가 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 생산조정제를 적극 추진키로 했다. 생산조정제는 벼를 심었던 논에 대신 사료작물이나 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 작물 재배 계획면적은 전국 5만ha의 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 변동직불금을 받은 농지에 10a 이외 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 수급 불안 품목은 제외된다. 농지의 경우도 이미 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.', '8억 투입, 고소천사벽화·자산마을에 색채 입혀 여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 마무리하며 색채와 빛의 도시를 완성했다. 시에 따르면 사업비 8억원이 투입된 컬러빌리지 사업은‘낮에는 색채, 밤에는 빛’을 주제로 지난해 착공에 들어갔다. 컬러빌리지는 색채를 이용한 경관개선사업으로 사업완료에 고소

In [42]:
# headline 부분 전처리 진행
clean_headlines = []
for s in data['headlines']:
  clean_headlines.append(preprocess_sentence(s, False))

print('Headlines 전처리 후 결과 : ', clean_headlines[:5])

<ipython-input-40-1b2392aabc35>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거


Headlines 전처리 후 결과 :  ["전라남도가 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 대신 사료작물이나 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다", '여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사벽화마을과 자산마을은 알록달록 색깔 옷을 입었고 사업 시행과 준공 과정에서도 주민들의 참여를 유도해 경관사업의 좋은 사례를 만들었다.', '전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단 목표 달성을 위한 새해 각오를 다졌다.', '광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 지역특화작목 중심으로 농업인 실용교육을 실시한다.', '올해 4월과 6월 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항해 전남의 관광지를 방문할 예정이다.']


In [43]:
data['text'] = clean_text
data['headlines'] = clean_headlines

data.replace('', np.nan, inplace=True)

data.isnull().sum()

text         0
headlines    0
dtype: int64

In [44]:
import matplotlib.pyplot as plt

text_len = [len(x.split()) for x in data['text']]
headlines_len = [len(x.split()) for x in data['headlines']]

print('텍스트 길이 최대: ', np.max(text_len))
print('텍스트 길이 최소: ', np.min(text_len))
print('텍스트 길이 평균: ', np.mean(text_len))
print('헤드라인 길이 최대: ', np.max(headlines_len))
print('헤드라인 길이 최소: ', np.min(headlines_len))
print('헤드라인 길이 평균: ', np.mean(headlines_len))

텍스트 길이 최대:  446
텍스트 길이 최소:  88
텍스트 길이 평균:  210.02611305652826
헤드라인 길이 최대:  91
헤드라인 길이 최소:  5
헤드라인 길이 평균:  23.307453726863432


In [47]:
# 텍스트, 헤드라인 최대 길이 설정
max_text_len = 400
max_headline_len = 60

# 최대 길이를 기준으로 나눴을때 최대 길이보다 짧은 데이터의 비율을 출력해주는 함수 정의
def below_threshold_len(max_len, list):
  count = 0
  for s in list:
    if len(s.split()) <= max_len:
      count += 1

  print(f'길이가 {max_len}보다 짧은 데이터의 비율 : {count/len(list)}')


# 비율 확인
below_threshold_len(max_text_len, data['text'])
below_threshold_len(max_headline_len, data['headlines'])

길이가 400보다 짧은 데이터의 비율 : 0.9970985492746374
길이가 60보다 짧은 데이터의 비율 : 0.9967983991995998


In [48]:
data['decoder_input'] = data['headlines'].apply(lambda x : 'starttoken ' + x)
data['decoder_output'] = data['headlines'].apply(lambda x : x + ' endtoken')

data.head()

,text,headlines,decoder_input,decoder_output
0,ha당 조사료 400만원…작물별 차등 지원 전라남도가 과잉문제를 근본적으로 해결하기...,전라남도가 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 대신 사료...,starttoken 전라남도가 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심...,전라남도가 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 대신 사료...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 여수시는 원도심 일대에서 추진된 컬러...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,starttoken 여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시...,여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발 전남드래곤즈는 지난 4일 구봉산 해맞이 행...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,starttoken 전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 ...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 지역특화작목 광양시는 오는 11일부터 24일까지 농업...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 지역특화작목 ...","starttoken 광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 지역특화작목 ..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 타이완의 크루즈관광객 4000...",올해 4월과 6월 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항해 ...,starttoken 올해 4월과 6월 차례에 걸쳐 타이완의 크루즈 관광객 4000여...,올해 4월과 6월 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항해 ...


In [49]:
encoder_input = np.array(data['text'])

decoder_input = np.array(data['decoder_input'])
decoder_output = np.array(data['decoder_output'])

# 데이터 섞어주기
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_output = decoder_output[indices]

# 샘플 추출
encoder_input[:5]

array(['광주송정역 주말 혼선 불가피…상경 수험생 불편 우려 역사에 열차 운행 중지 알림…장기화시 교통대란 보듯 도로서 파업 출정식 전국철도노동조합이 ‘4조 2교대’ 근무제 도입을 위한 인력 4천명 충원 등을 요구하며 무기한 총파업에 돌입한 20일 오후 철도노조 호남지방본부 노조원들이 광주 서구 광천버스터미널 인근 도로에서 파업 출정식을 갖고 있다. /김애리 기자 “파업이 장기화되면 피해를 받는 승객들입니다. 조속히 정상화되길 바랍니다.” 3년 만에 전국 철도노동조합이 무기한 총파업에 돌입한 가운데 파업 첫날인 20일 광주송정역사 출근길에는 혼란은 없었지만, 주말 사이 열차 운행 중단이 늘어날 것으로 보여 승객들의 불편은 불가피할 전망이다. 특히 대입 수시 논술과 면접고사 등을 앞두고 철도를 이용해 상경하려는 수험생들의 불편도 것으로 보인다. 이날 오전 광주송정역사. 출근길 혼선이 빚어질 것으로 보였지만 예상 외로 불편은 없었다. 열차 운행 중지 시간대가 늦은 오후에 편성돼 있어서다. 역사를 찾은 시민들은 파업 기간 운행 시간표를 사진으로 찍거나 안내데스크 직원에게 물어보는 파업을 대비하는 모습이었다. 역사 중앙에 위치한 대형 스크린에는 ‘노조 파업 일부 열차 운행 중지, 열차 이용에 불편을 드려 죄송합니다’라는 안내문이 눈에 띄었다. 역사 입구와 대합실 매표소 옆에 붙은 시간표는 선명한 ‘빨간줄’로 일부 열차들의 운행 중단을 알렸다. ‘합의 이행·직접 고용·비정규직 철폐’라고 적힌 조끼를 입은 역무원은 승객의 탑승을 돕고 있었으며 안내 데스크 직원들은 운행 취소와 파업 관련 문의가 쏟아지는 분주했다. 아들과 서울행 열차를 기다리던 박모씨는 “파업이 장기화되면 애꿎은 승객들만 불편을 겪는다. 서비스 이용과 물류 이송에도 차질이 생길 것”이라며 “노사가 조속히 합의해 이상의 불편은 가중되지 않았으면 한다”고 말했다. 용산행 열차에서 광주송정역으로 도착해 게이트를 지나던 송모씨는 “무기한 파업에 앞서 국민들에게 명확히 설명해 주고 그에 따른 공감대를 얻어야 

In [50]:
val_size = int(encoder_input.shape[0] * 0.2)

encoder_input_train = encoder_input[:-val_size]
decoder_input_train = decoder_input[:-val_size]
decoder_output_train = decoder_output[:-val_size]

encoder_input_test = encoder_input[-val_size:]
decoder_input_test = decoder_input[-val_size:]
decoder_output_test = decoder_output[-val_size:]

print('훈련 데이터의 개수 :', len(encoder_input_train))
print('훈련 레이블의 개수 :', len(decoder_input_train))
print('테스트 데이터의 개수 :', len(encoder_input_test))
print('테스트 레이블의 개수 :', len(decoder_input_test))

훈련 데이터의 개수 : 7996
훈련 레이블의 개수 : 7996
테스트 데이터의 개수 : 1999
테스트 레이블의 개수 : 1999


In [51]:
# 토크나이저 설정
src_tokenizer = Tokenizer()
src_tokenizer.fit_on_texts(encoder_input_train)



In [52]:
# 단어 빈도수 체크
threshold = 2
total_cnt = len(src_tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in src_tokenizer.word_counts.items():
  total_freq = total_freq + value

  if value < threshold:
    rare_cnt += 1
    rare_freq = rare_freq + value


print('단어 집합(vocabulary)의 크기 :', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 351921
등장 빈도가 1번 이하인 희귀 단어의 수: 225156
단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 126765
단어 집합에서 희귀 단어의 비율: 63.97913168012139
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 13.261695189987938


In [53]:
src_vocab = 20000
src_tokenizer = Tokenizer(num_words = src_vocab)
src_tokenizer.fit_on_texts(encoder_input_train)

# 정수 인코딩
# 텍스트 시퀀스를 정수 시퀀스로 변환
encoder_input_train = src_tokenizer.texts_to_sequences(encoder_input_train)
encoder_input_test = src_tokenizer.texts_to_sequences(encoder_input_test)

# 샘플 출력하여 확인
print(encoder_input_train[:3])

[[5511, 2619, 7023, 4137, 2224, 5844, 6605, 2144, 8793, 14846, 3801, 4499, 8, 968, 14847, 11677, 11, 6398, 13909, 17102, 10021, 348, 50, 12, 319, 264, 7558, 3801, 14848, 170, 1, 1828, 2, 560, 877, 1862, 18630, 289, 536, 171, 30, 13909, 17102, 10021, 32, 3801, 6823, 348, 11051, 2619, 2173, 6605, 2144, 2871, 6, 1542, 15922, 13106, 257, 19, 6606, 3385, 11, 208, 896, 15923, 6, 147, 18, 73, 18631, 6, 3386, 15922, 1559, 6605, 2144, 8793, 7559, 17103, 5684, 638, 814, 3004, 3801, 128, 2144, 11052, 13107, 14849, 17104, 917, 758, 474, 3801, 115, 6605, 2144, 8793, 6605, 7024, 1598, 1139, 7560, 917, 4823, 13108, 115, 2144, 13910, 4398, 6399, 2026, 3005, 7561, 1903, 5097, 2144, 3801, 26, 10022, 11678, 11053, 1598, 420, 8448, 2719, 8794, 317, 1862, 606, 15922, 101, 9, 15924, 7562, 123, 10023, 6824, 17105, 1134, 3439, 85, 3226, 317, 18632, 5512, 2385, 4724, 409, 2225, 100, 101, 13, 18, 50, 1419, 319, 1323, 534, 11054, 18633, 6032, 710, 4500, 10510, 7310, 17106, 165, 6210, 679, 1749, 3289, 3724, 115, 

In [54]:
tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(decoder_input_train)




In [55]:
threshold = 2
total_cnt = len(tar_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tar_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 70087
등장 빈도가 1번 이하인 희귀 단어의 수: 49917
단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 20170
단어 집합에서 희귀 단어의 비율: 71.22148187252986
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 25.491658027648263


In [56]:
tar_vocab = 10000
tar_tokenizer = Tokenizer(num_words=tar_vocab)
tar_tokenizer.fit_on_texts(decoder_input_train)
tar_tokenizer.fit_on_texts(decoder_output_train)

# 텍스트 시퀀스를 정수 시퀀스로 변환
decoder_input_train = tar_tokenizer.texts_to_sequences(decoder_input_train)
decoder_target_train = tar_tokenizer.texts_to_sequences(decoder_output_train)
decoder_input_test = tar_tokenizer.texts_to_sequences(decoder_input_test)
decoder_target_test = tar_tokenizer.texts_to_sequences(decoder_output_test)

# 잘 변환되었는지 확인
print('input')
print('input ',decoder_input_train[:5])
print('target')
print('decoder ',decoder_target_train[:5])

input
input  [[1, 5950, 1880, 2096, 73, 1189, 5951, 3969, 7889, 1269, 1881, 2328, 2913, 3970, 3971, 1115, 2914], [1, 19, 51, 174, 2329, 2915, 7890, 1116, 125, 1882, 4], [1, 114, 19, 65, 3369, 2595, 5952, 18, 2916, 71], [1, 2330, 5953, 34, 2596, 209, 680, 2331, 21, 4749, 3972, 4750, 3370, 7891, 417, 319, 1883, 4751, 178, 5954, 409], [1, 5955, 4752, 117, 7892, 7893, 1362, 2917, 4753, 1190, 1884, 4754, 397, 5956, 12, 7894, 126, 118, 11]]
target
decoder  [[5950, 1880, 2096, 73, 1189, 5951, 3969, 7889, 1269, 1881, 2328, 2913, 3970, 3971, 1115, 2914, 2], [19, 51, 174, 2329, 2915, 7890, 1116, 125, 1882, 4, 2], [114, 19, 65, 3369, 2595, 5952, 18, 2916, 71, 2], [2330, 5953, 34, 2596, 209, 680, 2331, 21, 4749, 3972, 4750, 3370, 7891, 417, 319, 1883, 4751, 178, 5954, 409, 2], [5955, 4752, 117, 7892, 7893, 1362, 2917, 4753, 1190, 1884, 4754, 397, 5956, 12, 7894, 126, 118, 11, 2]]


In [57]:
drop_train = [index for index, sentence in enumerate(decoder_input_train) if len(sentence) == 1]
drop_test = [index for index, sentence in enumerate(decoder_input_test) if len(sentence) == 1]

print('삭제할 훈련 데이터의 개수 :', len(drop_train))
print('삭제할 테스트 데이터의 개수 :', len(drop_test))

encoder_input_train = [sentence for index, sentence in enumerate(encoder_input_train) if index not in drop_train]
decoder_input_train = [sentence for index, sentence in enumerate(decoder_input_train) if index not in drop_train]
decoder_target_train = [sentence for index, sentence in enumerate(decoder_target_train) if index not in drop_train]

encoder_input_test = [sentence for index, sentence in enumerate(encoder_input_test) if index not in drop_test]
decoder_input_test = [sentence for index, sentence in enumerate(decoder_input_test) if index not in drop_test]
decoder_target_test = [sentence for index, sentence in enumerate(decoder_target_test) if index not in drop_test]

print('훈련 데이터의 개수 :', len(encoder_input_train))
print('훈련 레이블의 개수 :', len(decoder_input_train))
print('테스트 데이터의 개수 :', len(encoder_input_test))
print('테스트 레이블의 개수 :', len(decoder_input_test))

삭제할 훈련 데이터의 개수 : 1
삭제할 테스트 데이터의 개수 : 1
훈련 데이터의 개수 : 7995
훈련 레이블의 개수 : 7995
테스트 데이터의 개수 : 1998
테스트 레이블의 개수 : 1998


In [58]:
encoder_input_train = pad_sequences(encoder_input_train, maxlen=max_text_len, padding='post')
encoder_input_test = pad_sequences(encoder_input_test, maxlen=max_text_len, padding='post')
decoder_input_train = pad_sequences(decoder_input_train, maxlen=max_headline_len, padding='post')
decoder_target_train = pad_sequences(decoder_target_train, maxlen=max_headline_len, padding='post')
decoder_input_test = pad_sequences(decoder_input_test, maxlen=max_headline_len, padding='post')
decoder_target_test = pad_sequences(decoder_target_test, maxlen=max_headline_len, padding='post')

In [59]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 128
hidden_size = 256

# 인코더
encoder_inputs = Input(shape=(max_text_len,))

# 인코더의 임베딩 층
enc_emb = Embedding(src_vocab, embedding_dim)(encoder_inputs)

# 인코더의 LSTM 1
encoder_lstm1 = LSTM(hidden_size, return_sequences=True, return_state=True ,dropout = 0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

# 인코더의 LSTM 2
encoder_lstm2 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# 인코더의 LSTM 3
encoder_lstm3 = LSTM(hidden_size, return_state=True, return_sequences=True, dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# 디코더
decoder_inputs = Input(shape=(None,))

# 디코더의 임베딩 층
dec_emb_layer = Embedding(tar_vocab, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)

# 디코더의 LSTM
decoder_lstm = LSTM(hidden_size, return_sequences = True, return_state = True, dropout = 0.4)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = [state_h, state_c])

# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation = 'softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_outputs)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 400)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 400, 128)             2560000   ['input_3[0][0]']             
                                                                                                  
 lstm_4 (LSTM)               [(None, 400, 256),           394240    ['embedding_2[0][0]']         
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                            

In [60]:
import tensorflow as tf
import os
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [61]:
# 어텐션 층(어텐션 함수)
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# 어텐션의 결과와 디코더의 hidden state들을 연결
decoder_concat_input = Concatenate(axis = -1, name='concat_layer')([decoder_outputs, attn_out])

# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_concat_input)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 400)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 400, 128)             2560000   ['input_3[0][0]']             
                                                                                                  
 lstm_4 (LSTM)               [(None, 400, 256),           394240    ['embedding_2[0][0]']         
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                            

In [62]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
es = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
history = model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train,
                    validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
                    batch_size=256, callbacks=[es], epochs=10)

Epoch 1/10
32/32 [==============================] - 69s 2s/step - loss: 3.5253 - val_loss: 2.1137
Epoch 2/10
32/32 [==============================] - 61s 2s/step - loss: 2.2773 - val_loss: 2.0505
Epoch 3/10
32/32 [==============================] - 63s 2s/step - loss: 2.1994 - val_loss: 1.9733
Epoch 4/10
32/32 [==============================] - 64s 2s/step - loss: 2.1291 - val_loss: 1.9400
Epoch 5/10
32/32 [==============================] - 64s 2s/step - loss: 2.0966 - val_loss: 1.9282
Epoch 6/10
32/32 [==============================] - 65s 2s/step - loss: 2.0751 - val_loss: 1.8998
Epoch 7/10
32/32 [==============================] - 65s 2s/step - loss: 2.0627 - val_loss: 1.8902
Epoch 8/10
32/32 [==============================] - 65s 2s/step - loss: 2.0546 - val_loss: 1.8867
Epoch 9/10
32/32 [==============================] - 65s 2s/step - loss: 2.0503 - val_loss: 1.8837
Epoch 10/10
32/32 [==============================] - 65s 2s/step - loss: 2.0457 - val_loss: 1.8795


In [63]:
src_index_to_word = src_tokenizer.index_word # 원문 단어 집합에서 정수 -> 단어를 얻음
tar_word_to_index = tar_tokenizer.word_index # 요약 단어 집합에서 단어 -> 정수를 얻음
tar_index_to_word = tar_tokenizer.index_word # 요약 단어 집합에서 정수 -> 단어를 얻음


encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c])

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(hidden_size,))
decoder_state_input_c = Input(shape=(hidden_size,))

dec_emb2 = dec_emb_layer(decoder_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])


# 어텐션 함수
decoder_hidden_state_input = Input(shape=(max_text_len, hidden_size))
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# 디코더의 출력층
decoder_outputs2 = decoder_softmax_layer(decoder_inf_concat)

# 최종 디코더 모델
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [64]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    e_out, e_h, e_c = encoder_model.predict(input_seq)

     # <SOS>에 해당하는 토큰 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_word_to_index['starttoken']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition: # stop_condition이 True가 될 때까지 루프 반복

        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = tar_index_to_word[sampled_token_index]

        if(sampled_token!='endtoken'):
            decoded_sentence += ' '+sampled_token

        #  <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_token == 'endtoken'  or len(decoded_sentence.split()) >= (max_headline_len-1)):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 상태를 업데이트 합니다.
        e_h, e_c = h, c

    return decoded_sentence



In [65]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2text(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + src_index_to_word[i]+' '
    return temp

# 요약문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2summary(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_word_to_index['starttoken']) and i!=tar_word_to_index['endtoken']):
            temp = temp + tar_index_to_word[i] + ' '
    return temp

In [66]:
for i in range(50, 60):
    print("원문 :", seq2text(encoder_input_test[i]))
    print("실제 요약 :", seq2summary(decoder_input_test[i]))
    print("예측 요약 :", decode_sequence(encoder_input_test[i].reshape(1, max_text_len)))
    print("\n")

원문 : 판로 확대 소득 증대 기대 농협 전남지역본부는 지난 9일 지역본부 2층 대강당에서 무화과 참석한 가운데 전남 무화과 개최했다 전남농협 제공 농협 전남 대표 육성하기 위한 행보를 본격화하고 있다 농협 전남지역본부는 10일 “지난 9일 지역본부 2층 대강당에서 무화과 참석한 가운데 지난해 무화과 마무리하고 성공적인 사업 목표를 다짐하는 전남 무화과 개최했다”고 밝혔다 지난해 판매에 대한 빅데이터를 통해 분석하고 대해 공유하는 시간을 가졌다 2019년도 수립을 통해 산지 통한 농가 소득 증대 방안을 모색했다 지난해 선별 상향 소비자 맞는 개발 무화과 판로 확대 적극적으로 사업을 추진했다 결과 지난해 대비 15 성장한 원을 브랜드로 농가가 개별 출하 보다 이상의 농가 가격 제고 효과를 가져 것으로 평가됐다 김석기 본부장은 “전남 전남을 대표하는 우수 고품질 통해 농협은 고품질 생산 관리 체계를 중점을 두고 생산 대한 자부심을 전남의 내실 있는 전남의 대표 말했다 김동수 기자 김동수 기자 
실제 요약 : 농협 전남지역본부는 개최해 통한 분석하고 대해 공유하는 시간을 전남 대표 육성하기 위한 
1/1 [==============================] - 0s 35ms/step
예측 요약 :  지난 지난 지난 위해 위해 위해


원문 : 지자체 기관 유치 적극 지원 전남도가 오는 2022년 순천 내에 동부권 여수 광양 목포 시·군에 들어서는 전남도 관련 건립을 위해 지원에 나서기로 했다 못한 시·군에 대해서도 기관 유치 지역 현안사업 지원 방안을 검토할 방침이다 전남도는 12일 “올 하반기 동부권 통합청사 건립을 위한 행정절차를 마무리하고 2020년 실시설계를 착수해 하반기에 착공 2022년 예정이다”고 밝혔다 사업비는 부지에 건축 규모다 동부권 도민에게 보다 나은 제공하기 위한 동부권 통합청사 입지가 순천시 일원으로 선정됨에 이를 전남 발전의 되도록 건립 사업 추진에 속도를 했다 전남도는 여수에 ‘청소년 광양에 목포에 등의 건립을 추진중이다 전남도는 지